In [1]:
import pandas as pd
import numpy as np


df = pd.read_excel("../data/raw/Online Retail.xlsx")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

## UCI Online Retail

This dataset contains transactional records from a UK-based online retailer, spanning December 2010. Below is the variables table:

| Variable Name | Role     | Type         | Description                                                                 | Units     | Missing Values |
|---------------|----------|--------------|-----------------------------------------------------------------------------|-----------|----------------|
| `InvoiceNo`   | ID       | Categorical  | 6-digit number identifying each transaction. Prefix `'C'` indicates cancellation. | —         | No             |
| `StockCode`   | ID       | Categorical  | 5-digit code for each distinct product.                                    | —         | No             |
| `Description` | Feature  | Categorical  | Name or description of the product.                                        | —         | No             |
| `Quantity`    | Feature  | Integer      | Number of items purchased per transaction.                                 | Count     | No             |
| `InvoiceDate` | Feature  | DateTime     | Timestamp when the invoice was created.                                    | DateTime  | No             |
| `UnitPrice`   | Feature  | Continuous   | Price per unit of the product.                                             | Sterling  | No             |
| `CustomerID`  | Feature  | Categorical  | Unique identifier for each customer.                                       | —         | No             |
| `Country`     | Feature  | Categorical  | Country of residence for each customer.                                    | —         | No             |

## 1. Data Preprocessing
Let's check how the data looks like.

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Based on the non-null count shown below, we can see that the `Description` and `CustomerID` columns have null values.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


The output below shows the summary of the quantitative variables, we can see that there are some negative values within the columns.

In [ ]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


Let's also check for duplicate records within the dataset.

In [ ]:
print("Number of duplicate rows:", len(df[df.duplicated()]))

Number of duplicate rows: 5268


Let us drop the duplicated rows.

In [ ]:
df.drop_duplicates(inplace=True)

To address missing `Description` values, we will try to impute missing values through records with `StockCode` having a `Description`.
After this, the remaining missing values are given a placeholder.

In [ ]:
# Obtain records where 'Description' is not null
mapping = df[df['Description'].notna()]

# Keep only one row per 'StockCode'
mapping = mapping.drop_duplicates("StockCode")

# Create a mapping dictionary {StockCode: Description}
mapping_dict = mapping.set_index('StockCode')['Description'].to_dict()

df['Description'] = df.apply(
    lambda row: mapping_dict.get(row['StockCode'], row['Description']), axis=1
)

# Fill missing values  with a placeholder
df['Description'] = df['Description'].fillna('Unknown Item')

print("Number of missing 'Description' values after imputation:", df['Description'].isna().sum())

Number of missing 'Description' values after imputation: 0


Let us also explore the StockCodes not relevant for our analysis. Relevant StockCodes are those having 5 digits + possibly trailing letters.

In [ ]:
# Convert StockCode to string
df['StockCode'] = df['StockCode'].astype(str)

# Define regex pattern for valid stockcodes
pattern = r'^\d{5}[A-Za-z]*$'

# Flag rows where StockCode does NOT match the valid pattern
df['IsMetadata'] = ~df['StockCode'].str.match(pattern)

# Print unique invalid StockCodes
invalid_stockcodes = df.loc[df['IsMetadata'], 'StockCode'].unique()
print(invalid_stockcodes)


['POST' 'D' 'C2' 'DOT' 'M' 'BANK CHARGES' 'S' 'AMAZONFEE' 'DCGS0076'
 'DCGS0003' 'gift_0001_40' 'DCGS0070' 'm' 'gift_0001_50' 'gift_0001_30'
 'gift_0001_20' 'DCGS0055' 'DCGS0072' 'DCGS0074' 'DCGS0069' 'DCGS0057'
 'DCGSSBOY' 'DCGSSGIRL' 'gift_0001_10' 'PADS' 'DCGS0004' 'DCGS0073'
 'DCGS0071' 'DCGS0068' 'DCGS0067' 'DCGS0066P' 'B' 'CRUK']


Let us also flag canceled transactions (invoices starting with 'C').

In [ ]:
# Create IsCanceled flag for invoices starting with 'C'
df['IsCanceled'] = df['InvoiceNo'].astype(str).str.startswith('C')

Next, we will classify rows as 'MetaData', 'Canceled', or 'Sale'.

In [ ]:
def classify_transaction(row):
    if row['IsMetadata']:
        return 'Metadata'
    elif row['IsCanceled']:
        return 'Canceled'
    elif row['Quantity'] < 0:
        return 'Negative Qty'
    else:
        return 'Sale'

df['TransactionType'] = df.apply(classify_transaction, axis=1)

Let us now display the updated DataFrame. We can see that transactions are now marked for MetaData, cancellations, and its type.

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,IsMetadata,IsCanceled,TransactionType
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,False,False,Sale
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,False,False,Sale
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,False,False,Sale
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,False,False,Sale
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,False,False,Sale


Let us add a unique SaleID column to serve as the primary key.

In [ ]:
# Add a surrogate key: SaleID (starts from 1)
df['SaleID'] = range(1, len(df) + 1)

Let us also add a TotalPrice column via UnitPrice * Quantity.

In [ ]:
df['TotalPrice'] = df['UnitPrice'] * df['Quantity']

Finally, let us split the InvoiceDate into 'Date' and 'Timestamp' columns.

In [ ]:
df['Date'] = df['InvoiceDate'].dt.date
df['Timestamp'] = df['InvoiceDate'].dt.time


order = [
    'SaleID', 'InvoiceNo', 'StockCode', 'Description', 'Quantity', 
    'Date', 'Timestamp', 'InvoiceDate', 'UnitPrice', 'TotalPrice', 'CustomerID', 
    'Country', 'IsMetadata', 'IsCanceled', 'TransactionType'
]

df = df[order]
df.head()

,SaleID,InvoiceNo,StockCode,Description,Quantity,Date,Timestamp,InvoiceDate,UnitPrice,TotalPrice,CustomerID,Country,IsMetadata,IsCanceled,TransactionType
0,1,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,08:26:00,2010-12-01 08:26:00,2.55,15.30,17850.0,United Kingdom,False,False,Sale
1,2,536365,71053,WHITE METAL LANTERN,6,2010-12-01,08:26:00,2010-12-01 08:26:00,3.39,20.34,17850.0,United Kingdom,False,False,Sale
2,3,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,08:26:00,2010-12-01 08:26:00,2.75,22.00,17850.0,United Kingdom,False,False,Sale
3,4,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,08:26:00,2010-12-01 08:26:00,3.39,20.34,17850.0,United Kingdom,False,False,Sale
4,5,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,08:26:00,2010-12-01 08:26:00,3.39,20.34,17850.0,United Kingdom,False,False,Sale


## 2. Data Modeling
Given the flat DataFrame, we will model a star schema with a fact table and its dimension tables.

![Entity Relationship Diagram](../documentation/ERD.png)


First, let us create the dim_date table. We will create a DateID to serve as the key connecting both fact_sales and dim_date tables.

In [ ]:
dim_date = pd.DataFrame()
dim_date['DateID'] = df['Date'].apply(lambda x: int(x.strftime('%Y%m%d'))).drop_duplicates().reset_index(drop=True)
dim_date = dim_date.sort_values('DateID')

dim_date.head()

,DateID
0,20101201
1,20101202
2,20101203
3,20101205
4,20101206


Next, we will extract each date part to separate columns.

In [16]:
dim_date['Year'] = pd.to_datetime(df['Date']).dt.year
dim_date['Month'] = pd.to_datetime(df['Date']).dt.month
dim_date['Day'] = pd.to_datetime(df['Date']).dt.day
dim_date['Weekday'] = pd.to_datetime(df['Date']).dt.day_name()

date_order = [
    'DateID', 'Year', 'Month', 'Day', 'Weekday'
]

dim_date = dim_date[date_order]
dim_date.head()

,DateID,Year,Month,Day,Weekday
0,20101201,2010,12,1,Wednesday
1,20101202,2010,12,1,Wednesday
2,20101203,2010,12,1,Wednesday
3,20101205,2010,12,1,Wednesday
4,20101206,2010,12,1,Wednesday


Next, let us create the dim_product table.

In [17]:
dim_product = df[['StockCode', 'Description']].drop_duplicates().reset_index(drop=True)
dim_product = dim_product.sort_values('StockCode').reset_index(drop=True)

print(dim_product.head())

# Check for missing values in dim_product
print(dim_product.isna().sum())

print(len(dim_product['StockCode'].unique()), "unique StockCodes in dim_product")

  StockCode                  Description
0     10002  INFLATABLE POLITICAL GLOBE 
1     10080     GROOVY CACTUS INFLATABLE
2     10120                 DOGGY RUBBER
3    10123C        HEARTS WRAPPING TAPE 
4    10123G                 Unknown Item
StockCode      0
Description    0
dtype: int64
4070 unique StockCodes in dim_product


To ensure duplicated rows do not exist, we will standardize the StockCode values.

In [18]:
dim_product['StockCode'] = dim_product['StockCode'].str.strip().str.upper().drop_duplicates().reset_index(drop=True)

print(len(dim_product['StockCode'].unique()), "unique StockCodes in dim_product")

3959 unique StockCodes in dim_product


Also, we will remove any null values within the dim_product dataframe.

In [43]:
dim_product = dim_product.dropna(subset=['StockCode'])

print(len(dim_product['StockCode'].unique()), "unique StockCodes in dim_product")

3958 unique StockCodes in dim_product


Next, we will create the dim_customer table.

In [19]:
dim_customer = df[['CustomerID', 'Country']].drop_duplicates().reset_index(drop=True)
dim_customer = dim_customer.sort_values('CustomerID').reset_index(drop=True)

print(dim_customer.head())

# Check for missing values in dim_customer
print(dim_customer.isna().sum())

   CustomerID         Country
0     12346.0  United Kingdom
1     12347.0         Iceland
2     12348.0         Finland
3     12349.0           Italy
4     12350.0          Norway
CustomerID    9
Country       0
dtype: int64


The dim_customer dataframe still has missing values, we have to drop this for a seamless connection.

In [31]:
dim_customer = dim_customer.dropna(subset=['CustomerID'])
print(dim_customer.isna().sum())

print(len(dim_customer['CustomerID'].unique()), "unique CustomerIDs in dim_customer")

CustomerID    0
Country       0
dtype: int64
4372 unique CustomerIDs in dim_customer


Also, we will attempt to remove duplicated rows for a seamless connection.

In [32]:
dim_customer['CustomerID'] = dim_customer['CustomerID'].drop_duplicates().reset_index(drop=True)

print(len(dim_customer['CustomerID'].unique()), "unique CustomerIDs in dim_customer")

4372 unique CustomerIDs in dim_customer


Finally, we will create the fact_sales table.

In [34]:
fact_sales = df[
    ['SaleID', 'InvoiceNo', 'Quantity', 'UnitPrice', 'TotalPrice',
     'IsCanceled', 'IsMetadata', 'Date', 'Timestamp', 'TransactionType',
     'StockCode', 'CustomerID']].copy()

fact_sales['Date'] = pd.to_datetime(fact_sales['Date']).dt.date
fact_sales['DateID'] = fact_sales['Date'].apply(lambda x: int(x.strftime('%Y%m%d'))).reset_index(drop=True)


fact_sales.head(10000)

,SaleID,InvoiceNo,Quantity,UnitPrice,TotalPrice,IsCanceled,IsMetadata,Date,Timestamp,TransactionType,StockCode,CustomerID,DateID
0,1,536365,6,2.55,15.30,False,False,2010-12-01,08:26:00,Sale,85123A,17850.0,20101201.0
1,2,536365,6,3.39,20.34,False,False,2010-12-01,08:26:00,Sale,71053,17850.0,20101201.0
2,3,536365,8,2.75,22.00,False,False,2010-12-01,08:26:00,Sale,84406B,17850.0,20101201.0
3,4,536365,6,3.39,20.34,False,False,2010-12-01,08:26:00,Sale,84029G,17850.0,20101201.0
4,5,536365,6,3.39,20.34,False,False,2010-12-01,08:26:00,Sale,84029E,17850.0,20101201.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10202,9996,537233,12,2.95,35.40,False,False,2010-12-06,09:37:00,Sale,21844,17428.0,20101206.0
10203,9997,537233,24,1.95,46.80,False,False,2010-12-06,09:37:00,Sale,22812,17428.0,20101206.0
10204,9998,C537234,-20,1.95,-39.00,True,False,2010-12-06,09:40:00,Canceled,22653,16161.0,20101206.0
10205,9999,537235,156,8.50,1326.00,False,False,2010-12-06,09:45:00,Sale,22622,17381.0,20101206.0


## 3. Exporting the data

Let us start by checking the column headers for each dataframe.

In [23]:
print(dim_date.columns)
print(dim_product.columns)
print(dim_customer.columns)
print(fact_sales.columns)

Index(['DateID', 'Year', 'Month', 'Day', 'Weekday'], dtype='object')
Index(['StockCode', 'Description'], dtype='object')
Index(['CustomerID', 'Country'], dtype='object')
Index(['SaleID', 'InvoiceNo', 'Quantity', 'UnitPrice', 'TotalPrice',
       'IsCanceled', 'IsMetadata', 'Date', 'Timestamp', 'TransactionType',
       'StockCode', 'CustomerID', 'DateID'],
      dtype='object')


Next, we will export each dataframe as `.csv` files.

In [44]:
dim_date.to_csv('../data/cleaned/dim_date.csv', index=False)
dim_product.to_csv('../data/cleaned/dim_product.csv', index=False)
dim_customer.to_csv('../data/cleaned/dim_customer.csv', index=False)
fact_sales.to_csv('../data/cleaned/fact_sales.csv', index=False)
